In [1]:
%matplotlib inline
from effective_quadratures.parameter import Parameter
from effective_quadratures.polynomial import Polynomial
from effective_quadratures.indexset import IndexSet
from effective_quadratures.effectivequads import EffectiveSubsampling
from effective_quadratures.computestats import Statistics
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from effective_quadratures.utils import column, evalfunction

<h1> The Namesake: Effective Quadrature Subsampling

In this notebook we demonstrate what effective quadrature subsampling is and how it may be used to create a polynomial approximation for a computation model. We begin by defining a simple three-dimensional function
$$f(x_1, x_2, x_3) = exp(x_1 + x_2 + x_3)$$
where $x_1, x_2, x_3$ are uniformly distributed over the cube $x_1, x_2, x_3 
\in [-1,1]$. This will be our 'expensive' computational model. 

Lets first understand what exactly effective quadrature subsampling is and how it computes the pseudospectral coefficients. Have a look at the figure below. In (a) we have a matrix of multivariate orthonormal polynomials evaluated at a tensor grid of quadrature points (rows) and the underlying polynomial basis forms the columns. This basis can be formed from either a total order or hyperbolic cross basis. The "effectively subsampled quadrature points" are obtained by computing the QR column pivoting factorization of A transpose and only taking the first n columns of A transpose as shown in (b). We then compute the coefficients by solving this as a least squares problem. When approximating computational models with noise, one may need to eliminate a few extra columns to get a good least squares approximant; as in (c) and (d). We remark here that instead of opting for QR column pivoting, randomized sampling can also be opted for. However as we demonstrate in our paper, this is less efficient [and effective].

<img src='Matrices.png'>

To clarify the above, let us run though an example problem with a simple 

In [2]:
def exponential_function(x):
    return np.exp(x[0] + x[1] + x[2])

Declare the two inputs and their ranges

In [3]:
no_pts_x1 = 6
no_pts_x2 = 6
no_pts_x3 = 6
min_range = -1.0
max_range = 1.0
x1 = Parameter(lower=min_range, upper=max_range, points=no_pts_x1)
x2 = Parameter(lower=min_range, upper=max_range, points=no_pts_x2)
x3 = Parameter(lower=min_range, upper=max_range, points=no_pts_x3)
x1x2x3 = [x1, x2, x3]

We will opt for a hyperbolic cross basis, though one can also opt for a total order basis. Bear in mind that we are trying to escape the cost associated with a more conventional tensor grid or sparse grid basis. A hyperbolic cross takes in a *q* parameter. When *q=1* we have a total order basis. Note that the cardinality of the hyperbolic cross sets the maximum number of function evaluations required. 

In [4]:
q = 1.0
hyperbolic_cross = IndexSet("Hyperbolic basis", orders=[5,5,5], q=1.0)
maximum_number_of_evals = hyperbolic_cross.getCardinality()
print maximum_number_of_evals

AttributeError: 'IndexSet' object has no attribute 'getCardinality'

Lets plot this index set which has 47 terms:

In [ ]:
indexset = hyperbolic_cross.getIndexSet()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(column(indexset,0), column(indexset,1), column(indexset,2) , marker='o', s=80, color='red');
ax.set_xlabel('x1');
ax.set_ylabel('x2');
ax.set_zlabel('x3');
plt.title('Hyperbolic cross with q=0.6');


The class Effective_Subsampling has holds all the function calls required for effective quadrature subsampling. We begin by declaring an EffectiveSubsampling object that takes as inputs the uncertain parameters, the index set and a derivative flag. In this notebook as derivatives are not considered the derivative flag is set to 0.

In [ ]:
esq = EffectiveSubsampling(x1x2x3, hyperbolic_cross)
points = esq.getEffectivelySubsampledPoints(maximum_number_of_evals)
f = evalfunction(points, exponential_function)
x = esq.solveLeastSquares(maximum_number_of_evals, f)

Let's plot where these subsamples lie on our existing [no_pts_x1, no_pts_x2, no_pts_x3] tensor grid. 

In [ ]:
T = IndexSet("Tensor grid", [no_pts_x1, no_pts_x2, no_pts_x3])
uqProblem = Polynomial(x1x2x3, T)
tensor_pts, wts = uqProblem.getPointsAndWeights()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(tensor_pts[:,0], tensor_pts[:,1], tensor_pts[:,2], s=70, c='r', marker='o');
ax.scatter(points [:,0], points [:,1], points [:,2], s=150, facecolor='none', edgecolor='b', marker='o', lw=4);
ax.set_xlabel('x1');
ax.set_ylabel('x2');
ax.set_zlabel('x3');
plt.title('Effectively subsampled points');
plt.savefig('effective.png', format='png', dpi=500)

Now we solve the least squares problem:

In [ ]:
uq = Statistics(x, hyperbolic_cross)
mean = uq.getMean()
variance = uq.getVariance()

In [ ]:
print 'MEAN & VARIANCE'
print mean, variance

So how does this coefficient approximation fare with the full  tensor grid -- which involves 216 points?

In [ ]:
x_full, i, not_used = uqProblem.getPolynomialCoefficients(exponential_function)
tensor = Statistics(x_full, T)
mean_full = tensor.getMean()
variance_full = tensor.getVariance()

In [ ]:
print 'MEAN & VARIANCE'
print mean_full, variance_full

We can decrease the error by opting for a total order index. But recall the main point of this excercise was to escape the shear number of evaluations required from a tensor grid. 